In [6]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display
from scipy.ndimage import maximum_filter, gaussian_filter
import hashlib

In [ ]:


# Step 1: Load audio
y, sr = librosa.load('songs\\Run It Up - (Raag.Fm).mp3', sr=None)

# Step 2: Compute STFT
n_fft = 2048
hop_length = 512
S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
S_mag = np.abs(S)

# Step 3: Smooth spectrogram (optional, helps with noisy peaks)
S_smooth = gaussian_filter(S_mag, sigma=1.0)

# Step 4: 2D local maxima detection
neighborhood_size = 20
local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

# Step 5: Apply a threshold to keep only strong peaks
threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
peak_mask = local_max & (S_smooth >= threshold)

# Step 6: Get peak coordinates (freq_bin, time_bin)
peak_indices = np.argwhere(peak_mask)

# Step 7: Convert to time (sec) and frequency (Hz)
peak_freqs = peak_indices[:, 0] * (sr / n_fft)
peak_times = peak_indices[:, 1] * (hop_length / sr)

# Combine for (time, freq) tuples
constellation_map = list(zip(peak_times, peak_freqs))

# Print few peaks
print("Sample Peaks (time sec, frequency Hz):")
print(constellation_map[:10])

# Optional: Visualize peaks on spectrogram
"""plt.figure(figsize=(12, 6))
librosa.display.specshow(librosa.amplitude_to_db(S_mag, ref=np.max),
                         sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
plt.scatter(peak_times, peak_freqs, marker='x', color='red', s=10, label='Peaks')
plt.title('Spectral Peaks (Constellation Map)')
plt.colorbar(format='%+2.0f dB')
plt.legend()
plt.show()"""

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display
from scipy.ndimage import maximum_filter, gaussian_filter

# Step 1: Load audio
y, sr = librosa.load('songs\\Run It Up - (Raag.Fm).mp3', sr=None)

# Step 2: Compute STFT
n_fft = 2048
hop_length = 512
S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
S_mag = np.abs(S)

# Step 3: Smooth spectrogram (optional, helps with noisy peaks)
S_smooth = gaussian_filter(S_mag, sigma=1.0)

# Step 4: 2D local maxima detection
neighborhood_size = 20
local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

# Step 5: Apply a threshold to keep only strong peaks
threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
peak_mask = local_max & (S_smooth >= threshold)

# Step 6: Get peak coordinates (freq_bin, time_bin)
peak_indices = np.argwhere(peak_mask)

# Step 7: Convert to time (sec) and frequency (Hz)
peak_freqs = peak_indices[:, 0] * (sr / n_fft)
peak_times = peak_indices[:, 1] * (hop_length / sr)

# Combine for (time, freq) tuples
constellation_map = list(zip(peak_times, peak_freqs))

# Print few peaks
print("Sample Peaks (time sec, frequency Hz):")
print(constellation_map[:10])

# Optional: Visualize peaks on spectrogram
"""plt.figure(figsize=(12, 6))
librosa.display.specshow(librosa.amplitude_to_db(S_mag, ref=np.max),
                         sr=sr, hop_length=hop_length, x_axis='time', y_axis='hz')
plt.scatter(peak_times, peak_freqs, marker='x', color='red', s=10, label='Peaks')
plt.title('Spectral Peaks (Constellation Map)')
plt.colorbar(format='%+2.0f dB')
plt.legend()
plt.show()"""

In [ ]:
import hashlib

def generate_hashes(peaks, fan_value=5):
    """
    Generate landmark hashes from peaks.
    Each hash is made from (freq1, freq2, delta_time), anchored at time1.
    """
    hashes = []
    peaks = sorted(peaks)  # Sort by time

    for i in range(len(peaks)):
        t1, f1 = peaks[i]

        for j in range(1, fan_value + 1):
            if i + j < len(peaks):
                t2, f2 = peaks[i + j]

                delta_t = t2 - t1
                if 0 < delta_t <= 5.0:  # Limit max time delta for compact hashes
                    # Create hash string
                    hash_str = f"{int(f1)}|{int(f2)}|{int(delta_t * 100)}"
                    h = hashlib.sha1(hash_str.encode('utf-8')).hexdigest()[0:20]  # Shorten for space
                    hashes.append((h, t1))  # Store hash + anchor time

    return hashes



'# Generate hashes\nfingerprints = generate_hashes(constellation_map)\n\n# Show some hashes\nprint("Sample Fingerprints (hash, time):")\nfor h in fingerprints[:10]:\n    print(h)'

In [41]:

hashIndex={}
songsMeta={
    "Run It Up": {
        "title": "Run It Up",
        "artist": "hanumankind",
        "location": "songs\\Run It Up - (Raag.Fm).mp3"
    },
    "Wavy": {
        "title": "Wavy",
        "artist": "Karan Aujla",
        "location": "songs\\Wavy - Karan Aujla.mp3"
    }
}


for song in songsMeta.values():
    y, sr = librosa.load(song["location"], sr=None)

    # Step 2: Compute STFT
    n_fft = 2048
    hop_length = 512
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(S)

    # Step 3: Smooth spectrogram (optional, helps with noisy peaks)
    S_smooth = gaussian_filter(S_mag, sigma=1.0)

    # Step 4: 2D local maxima detection
    neighborhood_size = 20
    local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

    # Step 5: Apply a threshold to keep only strong peaks
    threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
    peak_mask = local_max & (S_smooth >= threshold)

    # Step 6: Get peak coordinates (freq_bin, time_bin)
    peak_indices = np.argwhere(peak_mask)

    # Step 7: Convert to time (sec) and frequency (Hz)
    peak_freqs = peak_indices[:, 0] * (sr / n_fft)
    peak_times = peak_indices[:, 1] * (hop_length / sr)

    # Combine for (time, freq) tuples
    constellation_map = list(zip(peak_times, peak_freqs))

    #get fingerprints
    fingerprint = generate_hashes(constellation_map)

    for h,t in fingerprint:
        if h not in hashIndex:
            hashIndex[h]=[(song["title"],t)]
        else:
            hashIndex[h].append((song["title"], t))

"""for fp in FingerPrints:
    print(f"Title: {fp['title']}, Artist: {fp['artist']}")
    print(fp["fingerprints"][:10])  # Show first 10 fingerprints for brevity
    print()  # Newline for better readability"""

'for fp in FingerPrints:\n    print(f"Title: {fp[\'title\']}, Artist: {fp[\'artist\']}")\n    print(fp["fingerprints"][:10])  # Show first 10 fingerprints for brevity\n    print()  # Newline for better readability'

In [ ]:
def SearchSong(songLocation):
    """
    Search for a song by title and artist.
    Returns the fingerprints if found, else None.
    """
    y, sr = librosa.load(songLocation, sr=None)

    # Step 2: Compute STFT
    n_fft = 2048
    hop_length = 512
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(S)

    # Step 3: Smooth spectrogram (optional, helps with noisy peaks)
    S_smooth = gaussian_filter(S_mag, sigma=1.0)

    # Step 4: 2D local maxima detection
    neighborhood_size = 20
    local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

    # Step 5: Apply a threshold to keep only strong peaks
    threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
    peak_mask = local_max & (S_smooth >= threshold)

    # Step 6: Get peak coordinates (freq_bin, time_bin)
    peak_indices = np.argwhere(peak_mask)

    # Step 7: Convert to time (sec) and frequency (Hz)
    peak_freqs = peak_indices[:, 0] * (sr / n_fft)
    peak_times = peak_indices[:, 1] * (hop_length / sr)

    # Combine for (time, freq) tuples
    constellation_map = list(zip(peak_times, peak_freqs))

    #get fingerprints
    fingerprint = generate_hashes(constellation_map)
    score={
    }
    for h,t_query in fingerprint:
        if h in hashIndex:
            for (song,t_db) in hashIndex[h]:
                delta= round(t_db-t_query, 2)
                score[(song,delta)] = score.get((song,delta), 0) + 1
    best_match = None
    best_score = 0

    for (song, delta), count in score.items():
        if best_match is None or count > best_score:
            best_match = song
            best_score = count
    print(songsMeta[best_match])

In [46]:
SearchSong("Testsongs\\Wavy - Karan Aujla (mp3cut.net).mp3")

{'title': 'Wavy', 'artist': 'Karan Aujla', 'location': 'songs\\Wavy - Karan Aujla.mp3'}


In [25]:
print(hashIndex)

{'e9d85168445a5bcb3077': ['Run It Up', 'Run It Up', 'Run It Up'], '032ac8b615011a35051a': ['Run It Up', 'Run It Up', 'Run It Up'], '566ba4f9efb53a5d4361': ['Run It Up', 'Run It Up'], '10323e3e8cdfdd4ca11e': ['Run It Up', 'Run It Up'], '29ba0003a1a396132092': ['Run It Up', 'Run It Up'], '680bbbc5d882d2218946': ['Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up'], '2dc9651022bbe0ca5fb3': ['Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up'], 'a49ed085f9ca42b3276f': ['Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up'], '66e37e51d2a0b7474220': ['Run It Up'], '393c42b968f3d9edf777': ['Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up'], '7b9c71a6821646bb89fc': ['Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up', 'Run It Up'], '5d4072f67968c927a097': 